In [39]:
from __future__ import division
import csv,sys
import numpy as np
import pandas as pd
import re
#import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time, locale
#Plot.ly
import plotly.plotly as py
from plotly.graph_objs import *
#from IPython import display as d
#import nvd3
from money import Money
### Librerias Propias
from load_data import obtener_provincia_cif
from load_data import obtener_tipo_cif
from load_data import obtener_tipo_cif_resumen
from load_data import string_to_timestamp
from load_data import listar_subvenciones_mes_grupo
from load_data import filtra_importes_grupo
from load_data import get_resumen_convocatoria
from load_data import get_programa_partida_pres

#from nvd3 import discreteBarChart
%load_ext autoreload
%autoreload 2

#reload(sys)  # Reload does the trick!
#sys.setdefaultencoding('UTF8')

#nvd3.ipynb.initialize_javascript(use_remote=True)

#locale.setlocale(locale.LC_ALL, 'es_ES')
#locale.setlocale(locale.LC_ALL, 'C')
#locale.format("%10.2f", 1234.567, grouping=True) 
#pd.options.display.float_format = '{:,.2f}'.format

# Static files
FILE_SUBVENCIONES = 'data/datos_subvenciones_2.csv'
FILE_AYUDAS = 'data/ayudas.json'
FILE_ELECCIONES = 'data/04_201105_1.csv'
FILE_MUNICIPIOS = 'data/smex99.csv'
FILE_ENTES_LOCALES = 'data/entes_locales.csv'
FILE_ENTES_JJAA = 'data/entes_jjaa.csv'
FILE_FUNCIONAL_PRESUPUESTO = 'data/partidas_presupuesto_gasto.csv'
FILE_CONVOCATORIAS_BOJA = "data/datos_resoluciones_2.csv"
# Referencias de partidas del presupuesto: http://www.juntadeandalucia.es/haciendayadministracionpublica/planif_presup/proy_presupuesto2014/presentacion/presentacion-5.pdf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Fuentes de Información
----------------------

El siguiente análisis está basado en las siguientes fuentes de datos:
* Subvenciones. Portal de Datos Abiertos de la Junta de Andalucia
* Ayudas. ????
* Municipios. Instituto de Estadística y Cartografía de Andalucía
* Elecciones. Instituto de Estadística y Cartografía de Andalucía
* Entes Municipales. Ministerio de Hacienda y Administraciones Públicas / Secretaría de Estado de Administraciones Públicas. 
* Entes Autonómicos. Ministerio de Hacienda y Administraciones Públicas / Secretaría de Estado de Administraciones Públicas. (también disponible en Consejería de Hacienda y Administraciones Públicas de la Junta de Andalucía).
* Anuario de la Caixa 2014.
* Presupuesto 2015 de la Junta de Andalucía

Se pueden descargar de:
* Datos Abiertos / Secretaría de Estado de Administraciones Públicas. http://www.minhap.gob.es/es-ES/Datos%20Abiertos/Paginas/CatalogoRISPSecretariadeEstadodeAdministracionesPublicas.aspx
* http://serviciostelematicos.minhap.gob.es/PubInvCCAA/secciones/FrmListadoEntes.aspx?cdcdad=AX9BV45F&tiporg2=
* http://www.anuarieco.lacaixa.comunicacions.com/java/X?cgi=caixa.le_menuGeneral.pattern
Referencias de partidas del presupuesto: http://www.juntadeandalucia.es/haciendayadministracionpublica/planif_presup/proy_presupuesto2014/presentacion/presentacion-5.pdf

In [40]:
#Read files
ayudas = pd.read_json(FILE_AYUDAS)
municipios = pd.read_csv(FILE_MUNICIPIOS, quotechar='"', sep=';', decimal=',', thousands='.')
elecciones = pd.read_csv(FILE_ELECCIONES, quotechar='"', sep=';', decimal=',', thousands='.')
entes_locales = pd.read_csv(FILE_ENTES_LOCALES, quotechar='"', sep=',')
entes_jjaa = pd.read_csv(FILE_ENTES_JJAA, quotechar='"', sep=',')
subvenciones = pd.read_csv(FILE_SUBVENCIONES, quotechar='"', sep='|', decimal=',').fillna('')
funcional = pd.read_csv(FILE_FUNCIONAL_PRESUPUESTO, quotechar='"', sep='|', decimal=',').fillna('')
convocatorias = pd.read_csv(FILE_CONVOCATORIAS_BOJA, sep='|', encoding='utf-8').fillna('')

El importe total de las subvenciones includas en los datos abiertos es:

In [41]:
total_subvenciones = subvenciones['importe'].sum() 
m = Money(amount = total_subvenciones, currency='EUR')
print(m.format('es_ES'))

1.038.724.228,81 €


# Mejora de la información

(Data wranling / Data Munging)

A continuación se procede a la mejora de la información de base con otras fuentes de información (de las ya cargadas),
o de la análisis de la ya disponible.

In [42]:
subvenciones['mes_concesion_timestamp'] = subvenciones['fecha_concesion'].apply(lambda x: string_to_timestamp(x))
subvenciones['fecha_concesion'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.strptime(x, '%d/%m/%y'))
subvenciones['mes_concesion'] = subvenciones['fecha_concesion'].apply(lambda x: x.strftime('%Y/%m'))

In [43]:
subvenciones['programa_de_partida'] = subvenciones.partida_pres.apply(get_programa_partida_pres)
subvenciones['tipo_beneficiario'] = subvenciones['id_beneficiario'].apply(obtener_tipo_cif)
subvenciones['tipo_beneficiario_res'] = subvenciones['id_beneficiario'].apply(obtener_tipo_cif_resumen)
subvenciones['tipo_beneficiario_cod'] = subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[0] if x != "" else "")
subvenciones['provincia'] = subvenciones['id_beneficiario'].apply(obtener_provincia_cif)
subvenciones['grupo_pres'] = subvenciones['programa'].apply(lambda x: ("%s" % x)[0] if x != "" else "")
subvenciones['provincia_cod_cif'] = subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[1:3])
subvenciones['convocatoria_res'] = subvenciones.convocatoria.apply(get_resumen_convocatoria)

In [44]:
subvenciones_por_fecha = subvenciones.groupby('fecha_concesion').sum()
subvenciones_por_mes = subvenciones.groupby('mes_concesion_timestamp').sum()
subvenciones_por_mes_grupo = subvenciones.groupby(['mes_concesion_timestamp','grupo_pres']).sum().reset_index()
subvenciones_por_finalidad = subvenciones.groupby('finalidad').sum().sort('importe',ascending=0).reset_index()
subvenciones_por_finalidad_beneficiario = subvenciones.groupby(['finalidad','tipo_beneficiario']).sum().sort('importe',ascending=0).reset_index()

In [45]:
elecciones['Nombre de Comunidad'] = elecciones['Nombre de Comunidad'].apply(lambda x: x.rstrip())
filas_andalucia = elecciones['Nombre de Comunidad'] == 'Andaluc\xc3\xada' #Andalucía


###¿Existen subvenciones no clasificadas?

El presupuesto marca el objetivo que se busca a muy grandes rasgos con las subvenciones. Por lo que es importante conocer si la información está o no disponible en el conjunto de datos

In [49]:
sum(subvenciones.programa=="")

6759

In [50]:
subvenciones[subvenciones.programa==""].groupby(['convocatoria_res'])\
.agg(['count','sum'])['importe']\
.sort(['sum','count'], ascending=[False,False]).head(10)

,count,sum
convocatoria_res,,
ORDEN DE 3 DE JULIO DE 2013,566,22088136.05
ORDEN DE 10 DE MARZO DE 2006,112,13654423.78
RESOL. 16 D FEBRERO D 2012,407,8347506.43
DECRETO-LEY 8/2013,2274,6023391.41
ORDEN DE 13 DE ABRIL DE 2010,126,4615182.18
ORDEN DE 28 DE MAYO DE 1999,1834,4581545.42
RESOL. DE 26 DE ABRIL DE 2013,252,3587719.90
RESOL. 1 D AGOSTO D 2011,189,3543660.60
RESOLUCION 10 DICIEMBRE 2013,100,2386186.29


Existen por tanto subvenciones que de las que no contamos con la función del presupuesto que le aplica.

### Ajustes manuales
* La *Orden de 28 de mayo de 1999* tiene como objetivo el sacrificio de animales, como no es posible obtenerla directamente del BOJA se fija contra el programa "71B", aunque también está relacionado con el "71C"
* La *Resolución del 16 de febrero de 2012* tiene como aplicación presupuestaria 01.12.00.01.00.773.48.610, por lo que debería ser 61O
* La *Orden de 10 de Marzo de 2006* no indica aplicación presupuestaria, pero revisando los conceptos funcionales del presupuesto parece aplicarle a [43A](http://www.juntadeandalucia.es/economiayhacienda/planif_presup/genero/informe/informe2015/informe-5-b-9.pdf).
* La *Resolución del 10 de diciembre de 2013* procederán de la partida 71P
* La *Orden de 7 de enero de 2009* hace referencia a varios programas: 31P, 41A, 41C, 41D, 41H, 41J, 41K.

In [59]:
programas_manuales = { 'ORDEN DE 28 DE MAYO DE 1999' : '71B',
                       'RESOL. 16 D FEBRERO D 2012' : '61O',
                       'ORDEN DE 10 DE MARZO DE 2006': '43A',
                       'ORDEN DE 3 DE JULIO DE 2013' : '71F',
                       'RESOLUCION 10 DICIEMBRE 2013' : '71P',
                       'SUBV. NOM. PARQUE CIENCIAS SUBVENCION NOMINATIVA PARQUE DE LAS CIENCIAS DE GRANADA, ANO 2014' : '44D',
                     }

# Asignacion de subvenciones nominativas sin asignar.
# 51B es donde el presupuesto reserva partidas para los metros
subvenciones = subvenciones.set_value(8088,'programa','51B')

In [60]:
convocatoria_partida = subvenciones[['convocatoria_res','programa_de_partida']]
programas_de_partida_presupuesto = ((convocatoria_partida.drop_duplicates()[convocatoria_partida.programa_de_partida != ""])[convocatoria_partida.convocatoria_res != ""]).set_index('convocatoria_res')['programa_de_partida'].to_dict()
programas_sin_convocatoria = ((convocatoria_partida[convocatoria_partida.programa_de_partida != ""])[convocatoria_partida.convocatoria_res == ""])['programa_de_partida'].to_dict()

In [61]:
programas_extraidos_boja = dict(zip(convocatorias.resolucion,convocatorias.programas))

indice_subvenciones_sin_programa = subvenciones[subvenciones.programa==""].index

programas_en_convocatoria = subvenciones['convocatoria'].str.contains('\s\d\d\D\s')
programas_en_convocatoria_res = subvenciones[programas_en_convocatoria]['convocatoria'].str.findall('\s\d\d\D\s').apply(lambda l: l[0])

programas_en_partida_pres = subvenciones['partida_pres'].str.contains('\d\d\D')
programas_en_partida_pres_res = subvenciones[programas_en_partida_pres]['partida_pres'].str.findall('\d\d\D').apply(lambda l: l[0])

for indice in indice_subvenciones_sin_programa:
    # Rescatamos el programa de otra columna que incluye la informacion
    if indice in programas_en_convocatoria_res:
        subvenciones.set_value(indice,'programa',programas_en_convocatoria_res[indice])
        continue
    if indice in programas_sin_convocatoria:
        subvenciones.set_value(indice,'programa',programas_sin_convocatoria[indice])
        continue        
    convocatoria_actual = subvenciones.iloc[indice].convocatoria_res
    if convocatoria_actual in programas_manuales:
        programa_manual = programas_manuales[convocatoria_actual]
        subvenciones.set_value(indice,'programa',programa_manual)
    elif convocatoria_actual in programas_de_partida_presupuesto:
        programa_de_partida = programas_de_partida_presupuesto[convocatoria_actual]
        subvenciones.set_value(indice,'programa',programa_de_partida)
    elif convocatoria_actual in programas_extraidos_boja:
        programa_boja = programas_extraidos_boja[convocatoria_actual]
        if programa_boja != '':
            #subvenciones.iloc[indice].programa = programas_boja
            subvenciones.set_value(indice,'programa',programa_boja)

In [88]:
print "%d programas de %d no han sido encontrados" % (sum(subvenciones.programa==""), len(subvenciones))

subvenciones_no_clasificadas = sum(subvenciones[subvenciones.programa==""].importe)
money_subvenciones_no_clasificadas = Money(amount = subvenciones_no_clasificadas, currency='EUR')
print "%s de ayudas de %s no han sido clasificadas" % (money_subvenciones_no_clasificadas.format('es_ES'), m.format('es_ES'))

808 programas de 71103 no han sido encontrados
8.806.726,16 € de ayudas de 1.038.724.228,81 € no han sido clasificadas


In [63]:
subvenciones[subvenciones.programa==""].groupby(['convocatoria_res'])\
.agg(['count','sum'])['importe']\
.sort(['sum','count'], ascending=[False,False]).head(10)

,count,sum
convocatoria_res,,
ORDEN DE 7 DE ENERO DE 2009,91,1981585.09
RESOL. DE 20 JUNIO 2013,6,884423.93
RESOL D 14 D JULIO D 2010,64,659337.63
RESOLUCION 25 NOVIEMBRE 2013,4,600000.00
ORDEN DE 19 DE ABRIL DE 2007,2,598790.52
ORDEN DE 31 DE JULIO DE 2009,4,562422.47
ORDEN D 26 D OCTUBRE D 2010,28,546030.74
ORDEN DE 23 DE OCTUBRE D 2009,10,451409.30
ORDEN D 12 D SEPTIEMBRE D 2008,3,310327.85


In [34]:
subvenciones.query(" convocatoria_res == '' and programa == ''") #['convocatoria'])



,id,base_reguladora,convocatoria,consejeria,organo_gestor,tipo_subvencion,programa,denominacion_programa,importe,finalidad,...,mes_concesion_timestamp,mes_concesion,programa_de_partida,grupo_pres,tipo_beneficiario,tipo_beneficiario_res,tipo_beneficiario_cod,provincia,provincia_cod_cif,convocatoria_res
36824,34247,CONVENIO SAS-FEAFES,SAS-FEAFES SUB.NOMINATIVA A FEAFES ANDALUCIA P...,SERVICIO ANDALUZ DE SALUD,SERVICIO DE APOYO AL S.A.S.,NOMINATIVA,,,150000,AYUDAS A FEDERACIONES DE APAS,...,1.409436e+12,2014/08,,,Asociaciones,Personas,G,NA,41,
58605,12340,CONVENIO SAS-PAUSAME,CONVENIO SAS-PAUSAME CONCEDER SUBVENCION A LA ...,SERVICIO ANDALUZ DE SALUD,SERVICIO DE APOYO AL S.A.S.,NOMINATIVA,,,75000,AYUDAS A FEDERACIONES DE APAS,...,1.401487e+12,2014/05,,,Asociaciones,Personas,G,NA,18,


Una vez se ha mejorado el dato del programa presupuestario se obtienen las distintas agregaciones del presupuesto

In [90]:
subvenciones['grupo_pres'] = subvenciones['programa'].apply(lambda x: ("%s" % x)[0] if x != "" else "")

# Análisis Exploratorio

A continuación con la información ya revisada y ampliada se puede proceder a intentar responder a cuestiones sobre la información con que se cuenta.

## ¿Cómo se distribuye el gasto a través del tiempo?


In [91]:
lista_mes_concesion=list(subvenciones['mes_concesion'].drop_duplicates())
lista_mes_concesion.reverse()
lista_mes_concesion

data = Data([
    Bar(
        x=lista_mes_concesion,
        y=list(subvenciones_por_mes['importe']),
        text=lista_mes_concesion,
        marker=Marker(
            color='rgb(142, 124, 195)'
        )
    )
])

layout = Layout(
    title='Distribución del gasto en subvenciones',
    font=Font(
        family='Raleway, sans-serif'
    ),
    showlegend=False,
    xaxis=XAxis(
        tickangle=-45
    ),
    yaxis=YAxis(
        zeroline=False,
        gridwidth=2
    ),
    bargap=0.05
)
fig = Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='bar-with-hover-text')
py.iplot(data, filename = 'evolucion-subveciones')

In [92]:
lista_meses_timestamp = subvenciones_por_mes_grupo['mes_concesion_timestamp'].unique()
lista_grupos = subvenciones_por_mes_grupo['grupo_pres'].unique()

In [93]:
def nueva_fila_mes_grupo(mes, grupo, importe = 0,id=0):
    new_data = { 'mes_concesion_timestamp' : [mes],
             'grupo_pres' : [grupo],
             'importe': [0], 
             'id': [id]}
    return pd.DataFrame(new_data)

In [94]:
subvenciones_mes_grupo_trabajo = subvenciones_por_mes_grupo.copy(deep=True)
contador_id = subvenciones_mes_grupo_trabajo['id'].argmax()

for mes in lista_meses_timestamp:
    for grupo in lista_grupos:
        query_mes_grupo = "mes_concesion_timestamp == %s & grupo_pres == '%s'" % (mes, grupo)
        resultados_query = subvenciones_mes_grupo_trabajo.query(query_mes_grupo)
        if len(resultados_query) == 0:
            contador_id += 1
            subvenciones_mes_grupo_trabajo = pd.concat([subvenciones_mes_grupo_trabajo,nueva_fila_mes_grupo(mes,grupo,id=contador_id)])


Subvenciones por área del presupuesto
---------------------

Utilizando el identificador del presupuesto podemos agrupar las ayudas por el xxx del presupuesto de la Junta de Andalucía y su distribución a lo largo del tiempo.

El presupuesto de las comunidades autónomas según su función se clasifican en:
* Área,
* Politica,
* Función y
* Programa

Los datos de subvenciones tienen detallado hasta la función, no existe detalle del programa.

In [95]:
subvenciones_por_grupo = subvenciones_por_mes_grupo.groupby('grupo_pres').sum()
areas_presupuesto = funcional.query("POLITICA=='' and  PROGRAMA ==''").set_index('AREA')['DESCRIPCION'].to_dict()
#total_subvenciones
#print list(subvenciones_por_grupo['importe'])
trace1 = Bar(
    x= areas_presupuesto.values(),
    y=list(subvenciones_por_grupo['importe']),
    name='SF Zoo'
)
#trace2 = Bar(
#    x=['giraffes', 'orangutans', 'monkeys'],
#    y=[12, 18, 29],
#    name='LA Zoo'
#)
data = Data([trace1])


layout = Layout(
    title='Subvenciones por área del presupuesto',
    font=Font(
        family='Raleway, sans-serif'
    ),
    showlegend=False,
    xaxis=XAxis(
        tickangle=-45
    ),
    yaxis=YAxis(
        zeroline=False,
        gridwidth=2
    ),
    bargap=0.05
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')


In [96]:
#areas_presupuesto
subvenciones_por_grupo

,mes_concesion_timestamp,id,importe
grupo_pres,,,
,1.686547e+13,221923801,8.495133e+07
1,9.889398e+12,2675677,5.003047e+06
2,5.632132e+12,119297,1.229375e+05
3,2.114096e+13,371419320,3.965595e+08
4,1.690210e+13,928131258,1.021258e+08
5,1.971321e+13,51749253,7.480284e+07
6,1.396217e+12,4672,3.500000e+05
7,1.971830e+13,921158832,2.883880e+08
8,1.408081e+13,30671746,8.642083e+07


In [68]:
#subvenciones_por_finalidad_beneficiario['finalidad'].drop_duplicates()

In [13]:
chart4_name = "Subvenciones por area"
chart4 = nvd3.stackedAreaChart(name=chart4_name,height=450,width=850,use_interactive_guideline=True, x_is_date=True)
chart4.set_containerheader("\n\n<h2>" + chart4_name + "</h2>\n\n")
xdata = list(subvenciones_por_mes_grupo.mes_concesion_timestamp.drop_duplicates())

for grupo in subvenciones_mes_grupo_trabajo.grupo_pres.order().drop_duplicates():
    chart4.add_serie(name=grupo, y=listar_subvenciones_mes_grupo(subvenciones_mes_grupo_trabajo,grupo), x=xdata)

chart4.margin_left = 100
chart4

Por Identificadores
-------------------

In [24]:
subvenciones['tipo_beneficiario'] = subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[0])
subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[0])#.drop_duplicates()

0        G
1        G
2        Q
3        G
4        G
5        G
6        G
7        G
8        G
9        G
10       Q
11       G
12       G
13       G
14       G
15       G
16       Q
17       G
18       7
19       P
20       P
21       P
22       G
23       P
24       P
25       P
26       P
27       P
28       P
29       P
        ..
71073    B
71074    3
71075    3
71076    B
71077    2
71078    3
71079    3
71080    3
71081    3
71082    2
71083    5
71084    2
71085    0
71086    2
71087    2
71088    3
71089    F
71090    2
71091    2
71092    B
71093    7
71094    5
71095    7
71096    2
71097    7
71098    7
71099    2
71100    7
71101    3
71102    P
Name: id_beneficiario, dtype: object

In [25]:
#subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[1:3])#.drop_duplicates()
subvenciones['id_beneficiario'].apply(obtener_tipo_cif)

0                                             Asociaciones
1                                             Asociaciones
2        Organismos autónomos, estatales o no, y asimil...
3                                             Asociaciones
4                                             Asociaciones
5                                             Asociaciones
6                                             Asociaciones
7                                             Asociaciones
8                                             Asociaciones
9                                             Asociaciones
10       Organismos autónomos, estatales o no, y asimil...
11                                            Asociaciones
12                                            Asociaciones
13                                            Asociaciones
14                                            Asociaciones
15                                            Asociaciones
16       Organismos autónomos, estatales o no, y asimil.

In [26]:
subvenciones.query("tipo_beneficiario_cod =='P' and provincia == 'Madrid'")['id_beneficiario']

Series([], Name: id_beneficiario, dtype: object)

In [27]:
subvenciones.groupby(['provincia']).count()
#subvenciones.query("provincia =='Teruel'")['id_beneficiario']

,id,base_reguladora,convocatoria,consejeria,organo_gestor,tipo_subvencion,programa,denominacion_programa,importe,finalidad,id_beneficiario,nombre_beneficiario,fecha_concesion,partida_pres,mes_concesion,grupo_pres,tipo_beneficiario,tipo_beneficiario_cod,provincia_cod_cif
provincia,,,,,,,,,,,,,,,,,,,
NA,71103,71103,71103,71103,71103,71103,64343,64343,71103,71103,71094,71103,71103,566,71103,71103,71103,71103,71103


In [28]:
subvencione

0   2015-03-27
1   2015-03-27
2   2015-03-27
3   2015-03-27
4   2015-03-27
Name: fecha_concesion, dtype: datetime64[ns]

0      31G
1      31G
2      31G
3      31G
4      31G
5      31G
6      31G
7      31G
8      31G
9      31G
10     31G
11     31G
12     31G
13     31G
14     31G
15     31G
16     31G
17     31G
18     32F
19     43A
20     43A
21     43A
22     71B
23     31B
24     31B
25     31E
26     31B
27     31G
28     31G
29     31G
      ... 
536    31B
537    11A
540    11A
541    75D
570    31G
571    31G
572    31G
573    31G
574    31G
575    31G
576    31G
577    31G
578    31G
579    31G
580    31G
581    31G
582    31G
583    31G
584    31G
585    31G
586    31G
587    31G
588    31G
589    31G
590    31G
591    31G
592    31G
593    31G
594    31G
596    32D
Name: partida_pres, dtype: object

0        3
1        3
2        3
3        3
4        3
5        3
6        3
7        3
8        3
9        3
10       3
11       3
12       3
13       3
14       3
15       3
16       3
17       3
18       3
19       4
20       4
21       4
22       7
23       3
24       3
25       3
26       3
27       3
28       3
29       3
        ..
71073    7
71074    7
71075    7
71076    7
71077    7
71078    7
71079    7
71080    7
71081    7
71082    7
71083    7
71084    7
71085    7
71086    7
71087    7
71088    7
71089    7
71090    7
71091    7
71092    7
71093    7
71094    7
71095    7
71096    7
71097    7
71098    7
71099    7
71100    7
71101    7
71102    3
Name: programa, dtype: object

In [169]:
subvenciones['partida_pres'].apply(lambda x: re.findall("\/(\d\d\D)\/",x))

0        [31G]
1        [31G]
2        [31G]
3        [31G]
4        [31G]
5        [31G]
6        [31G]
7        [31G]
8        [31G]
9        [31G]
10       [31G]
11       [31G]
12       [31G]
13       [31G]
14       [31G]
15       [31G]
16       [31G]
17       [31G]
18       [32F]
19       [43A]
20       [43A]
21       [43A]
22       [71B]
23       [31B]
24       [31B]
25       [31E]
26       [31B]
27       [31G]
28       [31G]
29       [31G]
         ...  
71073       []
71074       []
71075       []
71076       []
71077       []
71078       []
71079       []
71080       []
71081       []
71082       []
71083       []
71084       []
71085       []
71086       []
71087       []
71088       []
71089       []
71090       []
71091       []
71092       []
71093       []
71094       []
71095       []
71096       []
71097       []
71098       []
71099       []
71100       []
71101       []
71102       []
Name: partida_pres, dtype: object

{'ORDEN 28 MAYO 2014': '31B',
 'ORDEN 4 DE SEPTIEMBRE DE 2014': '31J',
 'ORDEN DE 10 DE MARZO DE 2006': '43A',
 'ORDEN DE 13 DE ABRIL DE 2010': '71B',
 'ORDEN DE 15 DE OCTUBRE DE 2013': '32D',
 'ORDEN DE 16 DE MAYO DE 2008': '71P',
 'ORDEN DE 20 DE JUNIO DE 2005': '31E',
 'ORDEN DE 21 DE OCTUBRE DE 2014': '71B',
 'ORDEN DE 21 JULIO DE 2005': '31B',
 'ORDEN DE 21 OCTUBRE DE 2014': '71B',
 'ORDEN DE 24 DE JULIO 2014': '32F',
 'ORDEN DE 24 JULIO DE 2014': '32F',
 'ORDEN DE 25 DE ABRIL DE 2014': '32G',
 'RESOLUCION DE 12 MARZO DE 2014': '41D',
 'RESOLUCION DE 24 DE JUNIO 2014': '31P',
 'RESOLUCI\xc3\x93N DE 19 DE DICIEMBRE DE 2008': '71P',
 'SUBV. NOMINATIVA COL. OFICIAL PSICOLOGOS AND. OCCIDENTAL 2015': '22B',
 'SUBVENCI\xc3\x93N NOMINATIVA GASTOS DE PERSONAL FREAAE 2015': '75D'}

In [221]:
convocatoria_partida = subvenciones[['convocatoria_res','programa_de_partida']]


{216: '31P', 537: '11A', 540: '11A'}